In [1]:
import pandas as pd
import numpy as np
from torchvision import datasets
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10 data augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

Using downloaded and verified file: ./data\cifar-10-python.tar.gz
Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# gettin the shape of the images
test,_ = next(iter(trainloader))

print(test.shape)

torch.Size([128, 3, 32, 32])


In [5]:
# building the model

import torch.nn as nn
from torch.optim import Adam

class MyConvBlock(nn.Module):
    def __init__(self):
        super(MyConvBlock, self).__init__()
        self.model = nn.Sequential(
        nn.Conv2d(3, 32,3),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
            
        nn.Conv2d(32, 64,3),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
            
        nn.Conv2d(64,128,3),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        
        nn.Flatten(),
            
        nn.Linear(128*2*2,64),
        nn.Linear(64,10)
        
        )
        

    def forward(self, x):
        return self.model(x)
    


model = MyConvBlock()
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()




In [6]:
labels = ["Airplane","Automobile","Cat","Deer","Dog","Frog","Horse","Ship","Truck"]


In [11]:
# Training the model

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}")



Epoch [1/10], Loss: 1.1537
Epoch [2/10], Loss: 1.0943
Epoch [3/10], Loss: 1.0512
Epoch [4/10], Loss: 1.0076
Epoch [5/10], Loss: 0.9824
Epoch [6/10], Loss: 0.9481
Epoch [7/10], Loss: 0.9326
Epoch [8/10], Loss: 0.9074
Epoch [9/10], Loss: 0.8923
Epoch [10/10], Loss: 0.8792


In [12]:
# Evaluating the model using the test dataset
# Testing the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {100 * correct / total:.2f}%')


Accuracy on the test set: 74.89%


In [13]:
# save the model

# Specify the path where you want to save the model
model_path = "model.pth"

# Save the model's state dictionary
torch.save(model.state_dict(), model_path)
print("Model saved successfully!")


Model saved successfully!
